In [3]:
import os
import sys
sys.path.append(os.path.pardir)
import numpy as np

import neo4j
from neo4j import GraphDatabase

from babelnet import BabelSynsetID, Language
from babelnet.data.relation import BabelPointer

from zerorpc import TimeoutExpired, LostRemote

2023-09-07 20:34:18,074 [babelnet.conf] INFO: Loaded configuration from ['/home/giovanni/unimore/2_graph_analytics/tesina/babeldist/remote/babelnet_conf.yml']
2023-09-07 20:34:18,079 [babelnet.api] INFO: BabelNet Remote Procedure Call API v1.1.0


In [4]:
URI = "bolt://localhost:7687"
AUTH = ("giovanni", "BabeldistGraph")

In [5]:
random_node_query = " MATCH (a:Synset) RETURN a.synsetID, rand() as r ORDER BY r LIMIT 1"

count_nodes_query = " MATCH (s:Synset) RETURN count(s) "
count_edges_query = " MATCH ()-[r:IS_A]->() RETURN count(r) "

shortestPath_query = """
MATCH (s1:Synset {synsetID: $synsetID_1})
MATCH (s2:Synset {synsetID: $synsetID_2})
MATCH p = shortestPath((s1)-[:IS_A*..12]-(s2))
RETURN p """

first_common_node_query = """
MATCH (s1:Synset {synsetID: $synsetID_1})
MATCH (s2:Synset {synsetID: $synsetID_2})
MATCH p = (s1)-[:IS_A*..5]->(common_node:Synset)<-[:IS_A*..5]-(s2) 
RETURN p, common_node """

max_length_taxonomy_query = """ 
MATCH p=(root:Synset {synsetID: $root_node})<-[:IS_A*1..12]-(child:Synset) 
WHERE NOT (child)<-[:IS_A]-()
RETURN p ORDER BY length(p) DESC LIMIT 1 """

In [6]:
root_node_id_str = 'bn:00062164n' # physical entity

In [7]:
driver = GraphDatabase.driver(URI, auth=AUTH)

In [8]:
def get_random_synset_id(driver):
    return driver.execute_query(random_node_query, result_transformer_=neo4j.Result.data)[0]['a.synsetID']

In [9]:
id1_str, id2_str = get_random_synset_id(driver), get_random_synset_id(driver) 
id1_str, id2_str

('bn:14978854n', 'bn:00014079n')

In [10]:
id1, id2 = BabelSynsetID(id1_str), BabelSynsetID(id2_str)
s1, s2 = id1.to_synset(), id2.to_synset()
s1, s2

(113951433__WIKI:EN:Oberrhaet_Formation, 47038672__Burundian#n#1)

In [11]:
def get_shortest_path(driver, s1_id: str, s2_id: str):
    return driver.execute_query(
        shortestPath_query, 
        {'synsetID_1': s1_id,
         'synsetID_2': s2_id},
        database_='neo4j',
        result_transformer_=neo4j.Result.data)

In [12]:
def get_shortest_path_length(driver, s1_id: str, s2_id: str):
    return get_shortest_path(driver, s1_id, s2_id)[0]['p'].count('IS_A')

In [13]:
def shortest_path_synsets_lemmas(shortest_path):
    for item in shortest_path:
        if type(item) is dict and 'synsetID' in item.keys():
            try:
                print(BabelSynsetID(item['synsetID']).to_synset().main_sense().full_lemma)
            except Exception as e:
                print(e.args[0])

In [14]:
def get_lcs_depth(driver, s1_id: str, s2_id: str):
    result = driver.execute_query(
        first_common_node_query,
        {'synsetID_1': s1_id,
         'synsetID_2': s2_id},
        database_='neo4j',
        result_transformer_=neo4j.Result.data)
    
    # TO-DO handle multiple results
    result = result[0]
    path, common_node = [s['synsetID'] for s in result['p'] if type(s) is dict], result['common_node']
    
    from_s1 = path[0:path.index(common_node['synsetID']) + 1]
    from_s2 = path[path.index(common_node['synsetID']):]
    return max(len(from_s1)-1, len(from_s2)-1)

In [15]:
def get_distance_from_root_node(driver, s1_id: str):
    return driver.execute_query(
        shortestPath_query,
        {'synsetID_1': s1_id,
         'synsetID_2': root_node_id_str},
        database_='neo4j',
        result_transformer_=neo4j.Result.data)[0]['p'].count('IS_A')

In [16]:
def get_max_taxonomy_length(driver, root_node_id: str):
    return driver.execute_query(
        max_length_taxonomy_query, 
        {'root_node': root_node_id_str},
        result_transformer_=neo4j.Result.data)[0]['p'].count('IS_A')

In [17]:
def wup_similarity(driver, s1_id: str, s2_id: str):
    lcs_depth = get_lcs_depth(driver, s1_id, s2_id)
    dist_s1_root = get_distance_from_root_node(driver, s1_id)
    dist_s2_root = get_distance_from_root_node(driver, s2_id)
    return lcs_depth / (dist_s1_root + dist_s2_root)

In [18]:
def lch_similarity(driver, s1_id: str, s2_id: str):
    d = get_max_taxonomy_length(driver, root_node_id_str)
    sp_length = get_shortest_path_length(driver, s1_id, s2_id)
    return -np.log(sp_length / (2 * d))

In [19]:
wup_similarity(driver, id1_str, id2_str)

0.5714285714285714

In [20]:
lch_similarity(driver, id1_str, id2_str)

0.9444616088408514

In [22]:
s1.main_gloss(), s2.main_gloss()

(The Oberrhaet Formation, also known as the Oberrhaet Limestone, is a Late Triassic geological formation in Austria.,
 A native or inhabitant of Burundi)

In [21]:
driver.close()